# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [42]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np
from collections import Counter
from pprint import pprint

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
 
lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))

#Get number of words in line
word_count_line_raw = [len(line.split()) for line in lines]
#Delete 0 left after line splitting
word_count_line = list(filter(lambda a: a != 0, word_count_line_raw))

print('Average number of words in each line: {}'.format(np.average(word_count_line)))
print('Median: {}'.format(np.median(word_count_line)))
print('Mode: {}'.format(helper.mode(word_count_line)))
print('Variance: {}'.format(np.var(word_count_line)))
print('Standard deviation: {}'.format(np.std(word_count_line)))

#Find occurrences (Percent of same number of words in line)
occ = Counter(word_count_line)
occ_list = [(i, occ[i], occ[i] / len(word_count_line) * 100.0) for i, count in occ.most_common()] 
print('Occurrences')
pprint (occ_list)
 
print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 11.088582100483375
Median: 8.0
Mode: {2, 4}
Variance: 141.06791855496303
Standard deviation: 11.877201629801652
Occurrences
[(4, 4517, 8.270470191885163),
 (2, 4517, 8.270470191885163),
 (6, 4499, 8.237512816756993),
 (5, 4451, 8.14962648308188),
 (3, 4237, 7.757799912113667),
 (7, 3919, 7.17555295151604),
 (8, 3404, 6.232605829793467),
 (9, 3098, 5.672330452614618),
 (10, 2524, 4.6213563790830525),
 (11, 2264, 4.145305405009521),
 (12, 1871, 3.425736048044529),
 (13, 1633, 2.9899663102387577),
 (14, 1455, 2.6640544895268783),
 (15, 1221, 2.2356086128607),
 (16, 1162, 2.1275816610517064),
 (17, 1015, 1.8584297641716714),
 (18, 850, 1.5563204921634686),
 (19, 777, 1.4226600263659002),
 (20, 652, 1.1937893657536254),
 (21, 588, 1.0766075875201406),
 (22, 577, 1.0564669693862605),
 (23, 507, 0.9282993994433865),
 (24, 431, 0.7891460377911235),
 (25, 398, 0.728724183389483)

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    vocab = tuple(set(text))
    int_to_vocab = dict(enumerate(vocab))
    vocab_to_int = {ch: ii for ii, ch in int_to_vocab.items()}
   # print(text,'#', vocab,'#', int_to_vocab,'#', vocab_to_int)
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    symbols_dict = {
                '.' : '||period||',
                ',' : '||comma||',
                '"' : '||quotation_mark||',
                ';' : '||semicolon||',
                '!' : '||exclamation_mark||',
                '?' : '||question_mark||',
                '(' : '||left_parentheses||',
                ')' : '||right_parentheses||',
                '-' : '||dash||',
                '\n' : '||return||',
             }
    
    return symbols_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    features, targets = [], []
    
    # Keep only enough words to make full batches
    for index in range(0, len(words)-sequence_length):
        sequence_end = index+sequence_length
        features_sequence_slice = words[index:sequence_end]
        features.append(features_sequence_slice)
        targets.append(words[sequence_end])
    
    data = TensorDataset(torch.from_numpy(np.asarray(features, dtype=np.int64)), torch.from_numpy(np.asarray(targets, dtype=np.int64)))
    data_loader = DataLoader(data, shuffle=False, batch_size=batch_size)
    return data_loader  
        

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# Test dataloader

test_text = range(1000)
t_loader = batch_data(test_text, sequence_length=16, batch_size=512)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.__next__()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([512, 16])
tensor([[  0,   1,   2,  ...,  13,  14,  15],
        [  1,   2,   3,  ...,  14,  15,  16],
        [  2,   3,   4,  ...,  15,  16,  17],
        ...,
        [509, 510, 511,  ..., 522, 523, 524],
        [510, 511, 512,  ..., 523, 524, 525],
        [511, 512, 513,  ..., 524, 525, 526]])

torch.Size([512])
tensor([ 16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
         44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
         58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
         72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
         86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 13

---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [40]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # Set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)

        # Embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        gru_output, hidden = self.gru(embeds, hidden)
        
        gru_output = gru_output.contiguous().view(-1, self.hidden_dim)
        output = self.fc(gru_output)
        
        # Reshape to set batch_size first
        output = output.view(batch_size, -1, self.output_size)
        output = output[:, -1]
        
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        # Initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim)).cuda()
        else:
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim))
        return hidden

#Test fails because GRU model requires to use hidden as Variable instead of tuple   
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

AssertionError: Wrong hidden state size. Expected type (2, 50, 10). Got type torch.Size([50, 10])

Unit Test Function Parameters
Input Size: 20
Output Size: 20
Hidden Dim: 10
N Layers: 2
Batch Size: 50
Sequence Length: 3
Input: tensor([[ 1,  4, 15],
        [ 0, 11,  2],
        [13, 17,  6],
        [10, 14, 11],
        [ 0, 14,  5],
        [ 7, 17,  8],
        [11,  5, 16],
        [ 4,  6,  2],
        [ 8,  9,  6],
        [ 0,  2,  5],
        [ 6, 14, 14],
        [ 3,  7, 17],
        [ 2, 16, 15],
        [15,  1, 10],
        [13, 14, 15],
        [14,  3,  2],
        [ 9, 14, 13],
        [12, 13, 13],
        [10,  0,  5],
        [12,  6, 10],
        [17, 17, 13],
        [12,  8,  2],
        [12, 13, 10],
        [ 3, 19,  1],
        [17, 14, 14],
        [12, 11, 13],
        [ 7,  6,  9],
        [ 3,  3,  8],
        [ 1,  9, 18],
        [ 8,  9, 16],
        [ 2,  3, 11],
        [13, 10,  8],
        [ 8,  3,  9],
        [ 9, 17,  2],
        [ 6, 15,  1],
  

### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [41]:
def forward_back_prop(rnn, optimizer, criterion, inputs, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # Move data to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
        inputs, target = inputs.cuda(), target.cuda()

    # Zero accumulated gradients
    rnn.zero_grad()    
    
    # Get the output from the model
    output, hidden = rnn(inputs, hidden.detach())
    
    # Perform backpropagation and optimization
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
# Test fails because GRU model requires to use hidden as Variable instead of tuple   
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

AssertionError: Returned hidden state is the incorrect size.

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [13]:
# Data params
# Sequence Length
sequence_length = 16  # of words in a sequence
# Batch Size
batch_size = 512

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [14]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int) 
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 200
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.426004745483398

Epoch:    1/10    Loss: 4.66904469871521

Epoch:    1/10    Loss: 4.322114735603333

Epoch:    2/10    Loss: 4.108641701245887

Epoch:    2/10    Loss: 3.943017961025238

Epoch:    2/10    Loss: 3.8538445663452148

Epoch:    3/10    Loss: 3.769163883921271

Epoch:    3/10    Loss: 3.6797442669868468

Epoch:    3/10    Loss: 3.60221563911438

Epoch:    4/10    Loss: 3.546202252174645

Epoch:    4/10    Loss: 3.4802679982185363

Epoch:    4/10    Loss: 3.410179849147797

Epoch:    5/10    Loss: 3.367472914351286

Epoch:    5/10    Loss: 3.312697371959686

Epoch:    5/10    Loss: 3.252104464530945

Epoch:    6/10    Loss: 3.2197205384143923

Epoch:    6/10    Loss: 3.167750141620636

Epoch:    6/10    Loss: 3.11646643781662

Epoch:    7/10    Loss: 3.0929596825751333

Epoch:    7/10    Loss: 3.0457633085250855

Epoch:    7/10    Loss: 3.003271558761597

Epoch:    8/10    Loss: 2.985839001252002

Epoch:    8/10    Loss:

c:\programdata\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 

Decided to try GRU, model's architecture requires to return hidden as Variable instead of tuple used for LSTM. With chosen hyper-parameters was possible to reach 2.75 loss in 10 epochs and generate some silly dialogs.

- `sequence_length`: calculated mean and standard deviation show narrow spread. Motivation to choose 16 was to pass a sequence a little bit longer than average sentence for more context.
- `batch_size`: started with 1024, bigger batch sizes could cause GPU memory issues. I think the bigger batch size the better and longer context rich text could be generated. Decided to keep 512 for efficient training.
- `num_epochs`: was set to 10 to pass initial requirement, but seems network could go lower.
- `learning_rate`: initially started with 0.1, empirically found the best value of 0.001 for fastest convergence. 
- `embedding_dim`: usual value for pretrained embeddings is 300, to keep training efficient decreased it to 200, decreasing lower would increase loss decay.
- `hidden_dim`: based on coursework example decided to choose 200, future plan is to experiment with hidden_dim higher or lower embedding_dim.
- `n_layers`: seems 2 is working best with this model.
- `dropout`: model showed much better results without dropout.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [38]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

c:\programdata\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:42: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


elaine:. i'll see you later.

kramer: hey, jerry. i got gonorrhea from the frogger.

elaine: well, i guess i can unwind.

george: what do you think?

man: it's walter's.

elaine: i can't do that.

kramer: no, no.

jerry: i can't go.

elaine: hey, jerry, crank up.

jerry: all right, brace this.

kramer: oh, no, no, it was mara conchita alonso.

elaine: oh, no, i can't. i haven't spoken to california.

jerry: you know, the new yorker, the new yorker--

hoyt: geraldo!

hoyt: so, what are you doing with this plane?

jerry: i don't know, but i can't tell that, it's a long island.

jerry: what?

george: what is he doing?

kramer: well, i was thinking.

jerry: you don't want to know where i do.

george: well, i think it's a good one.

jerry: hey, what do you think?

george: i can't go anywhere, get out!

jerry: all right, brace yourself, let's see if i could get my shoes off, but it was a little slow. i mean, what would you do?

sales woman: i don't think it's a big long term.

vandelay: obje

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [39]:
# save script to a text file

#naming convention generated_script_<sequence_length>_<batch_size>_<num_epochs>_<learning_rate>_<emb_dim>_<hidden_dim>_<n_layers>.txt

f =  open("generated_script_elaine.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.